# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 53.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=13e88360d0a1ad617a2674e3bb417144fad25c270de7350f3a6d78d00aa301bc
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [7]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [8]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [9]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [63]:
# YOUR CODE HERE
joined = orders.join(products, orders.product_id==products.product_id)
orders_joined = orders.groupBy(orders.order_id).agg(collect_list('product_id').alias('basket'))\
                    .select(['order_id', 'basket'])
orders_joined.show()

+--------+--------------------+
|order_id|              basket|
+--------+--------------------+
|       1|[49302, 11109, 10...|
|      96|[20574, 30391, 40...|
|     112|[27104, 21174, 41...|
|     218|[1194, 5578, 3815...|
|     456|[18196, 34229, 48...|
|     473|[20082, 24852, 47...|
|     631|[21137, 27344, 42...|
|     762|[21137, 41220, 15...|
|     774|[47482, 43335, 16...|
|     844|[14992, 21405, 11...|
|     904|       [8013, 46149]|
|     988|[45061, 28464, 12...|
|    1032|[11063, 26165, 27...|
|    1077|[13176, 39922, 52...|
|    1119|[6046, 47626, 184...|
|    1139|[24852, 21137, 34...|
|    1143|[42719, 42097, 36...|
|    1145|[24852, 1465, 280...|
|    1275|[6046, 48679, 545...|
|    1280|[48186, 49235, 23...|
+--------+--------------------+
only showing top 20 rows



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [65]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="basket", minSupport=0.01, minConfidence=0.5)

model = fpGrowth.fit(orders_joined)

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [66]:
# YOUR CODE HERE
model.freqItemsets.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------+-----+
|         items| freq|
+--------------+-----+
|        [5025]| 1445|
|       [11777]| 1493|
|       [37067]| 2332|
|       [20114]| 1899|
|       [19057]| 2891|
|       [22035]| 1993|
|       [43961]| 2460|
|       [26209]| 6033|
|[26209, 47626]| 1595|
|[26209, 24852]| 1331|
|       [43352]| 3279|
|       [38739]| 1633|
|       [20995]| 1361|
|       [27344]| 1788|
|       [19660]| 2225|
|       [28985]| 2627|
|        [4605]| 3762|
|       [21137]|10894|
|[21137, 13176]| 3074|
|[21137, 24852]| 2174|
+--------------+-----+
only showing top 20 rows



Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [69]:
# YOUR CODE HERE

fpGrowth = FPGrowth(itemsCol="basket", minSupport=0.001, minConfidence=0.5)

model = fpGrowth.fit(orders_joined)
model.freqItemsets.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------+----+
|         items|freq|
+--------------+----+
|       [26317]| 370|
|        [6532]| 259|
|       [46676]| 993|
|[46676, 47766]| 137|
|[46676, 13176]| 157|
|[46676, 21903]| 143|
|[46676, 24852]| 258|
|       [26940]| 730|
|[26940, 21137]| 185|
|[26940, 13176]| 263|
|[26940, 21903]| 136|
|[26940, 47209]| 164|
|       [44661]| 365|
|       [44750]| 140|
|        [8087]| 283|
|        [5025]|1445|
| [5025, 26209]| 206|
| [5025, 47766]| 133|
| [5025, 28842]| 187|
| [5025, 47626]| 229|
+--------------+----+
only showing top 20 rows



To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [ ]:
# YOUR CODE HERE


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!